In [10]:
import torch

# 현재 Setup 되어있는 device 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print ('Available devices ', torch.cuda.device_count())
print ('Current cuda device ', torch.cuda.current_device())
print(torch.cuda.get_device_name(device))

Available devices  4
Current cuda device  3
Tesla V100-PCIE-32GB


In [11]:
# GPU 할당 변경하기
GPU_NUM = 3 # 원하는 GPU 번호 입력
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device) # change allocation of current GPU
print ('Current cuda device ', torch.cuda.current_device()) # check

# Additional Infos
if device.type == 'cuda':
    print(torch.cuda.get_device_name(GPU_NUM))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(GPU_NUM)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(GPU_NUM)/1024**3,1), 'GB')


Current cuda device  3
Tesla V100-PCIE-32GB
Memory Usage:
Allocated: 0.9 GB
Cached:    6.0 GB


In [22]:
import random
import math
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import MinMaxScaler, StandardScaler
# ======================================
# Prepare Data
def make_batch(data, batch_size, window_size, shuffle=True):
    window_list = []
    for i in range(len(data) - window_size - 1):
        window = data[i: i + window_size]
        window_list.append(window)
#데이터에서 사이즈씩 뽑아서 리스트를 만들어줌...

    if shuffle:
        random.shuffle(window_list)
#셔플은 기본적으로 필요함

#윈도우 리스트는?
#윈도우 사이즈 - 열 숫자? 
#배치 사이즈 - 한번에 넘겨주는 데이터 사이즈-몇개의윈도우를 한번에 넘겨줄것인가.
    n_batch = math.ceil(len(window_list) / batch_size)
    batch_list = []
    for i in range(n_batch): #100에 10개씩이면 10번만
        batch = window_list[i*batch_size: (i+1)*batch_size] #0~9
        batch_list.append(batch)
    batch_list = np.array(batch_list)

    return batch_list

data = pd.read_csv("./10feature_cpu.csv")
col = list(map(str, data.columns))
data = data[col[:-1]]
data = data.to_numpy()

scaler = StandardScaler()
dscaler = scaler.fit(data)
data = dscaler.transform(data)

# ========================================
# Modeling
class SequenceModel(nn.Module): # 아웃풋이 4라는건 4개의 값을 모두 예측? #히든 사이즈의 정의 # 넘레이어..? 보통 사용하는 숫자..찾아봐야 함..
    def __init__(self, input_size=4, output_dim=4, hidden_size=256, num_layers=1):
        super(SequenceModel, self).__init__()
        self.lstm = nn.LSTM(input_size=input_size,
                            hidden_size=hidden_size,
                            num_layers=num_layers,
                            batch_first=True)
        self.scaler_bias = nn.Parameter(torch.ones(input_size, requires_grad=True))
        self.scaler = nn.Parameter(torch.ones(input_size, requires_grad=True))
        self.linear = nn.Linear(hidden_size, output_dim)

    def forward(self, x):
        x = (x + self.scaler_bias) * self.scaler #식은 어디서 확인 가능..?
        zs, hidden = self.lstm(x)
        z = zs[:, -1]
        v = self.linear(zs)
        return v, z


# ======================================
# Training
window_size = 80
batch_size = 2048
hidden_size = 20
use_cuda = True

model = SequenceModel(input_size=data.shape[-1],
                      output_dim=data.shape[-1],
                      hidden_size=hidden_size,
                      num_layers=5)

if use_cuda:
    model.cuda()

optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()

n_epoch = 50000
ema_loss = None
alpha = 0.1
verbose_interval = 100

for epoch_i in range(n_epoch):

    batch_list = make_batch(data, batch_size, window_size+1)
    for batch_i, batch in enumerate(batch_list):
        optimizer.zero_grad()

        batch = np.array(batch)
        batch_input = batch[:, :-1, :] #3차?
        batch_output = batch[:, 1:, :]

        batch_input = torch.tensor(batch_input, dtype=torch.float32)
        batch_output = torch.tensor(batch_output, dtype=torch.float32)

        if use_cuda:
            batch_input = batch_input.cuda()
            batch_output = batch_output.cuda()


        v, _ = model(batch_input)

        loss = loss_fn(v, batch_output) #output, Target 로스값 계산. 해서 다시넣는 과정..

        loss.backward()
        optimizer.step()
        
        if ema_loss is None:
            ema_loss = loss.item()
        ema_loss = loss.item() * alpha + (1.-alpha) * ema_loss

    if epoch_i % verbose_interval == 0:
        print(f"{epoch_i}th epoch, loss: {ema_loss}")
        #print(v.shape)
        #print(v)
        #print()

#만번 학습시킨다음에 마지막 v값 한번 봐보자
#윈도우 배치 히든 n_lay
#5 512 10 10 8.8
#20 2048 20 20 - 20 20 이 문제가있는듯 -> num_layer가 너무 높으면 학습이안됨. 히든사이즈는 ok 근데또 num_l 1보단 10이 성능이좋음 5는 더좋네(만일때 3프로대진입) 
#" 10 10 6.8
# n이 3일때 1.8퍼까지. 7은 2퍼  2는 3퍼 6은 1.95 !4가 1.6퍼 & 5도 1.6퍼  8은 2.2..
#n5에 윈도우 40주니 1.1퍼 윈60 1퍼  윈 80 1퍼 윈100 0.9퍼


0th epoch, loss: 1.0275311470031738
100th epoch, loss: 0.35861801330815724
200th epoch, loss: 0.20775485351879144
300th epoch, loss: 0.1784325284490592
400th epoch, loss: 0.1674838155822686
500th epoch, loss: 0.161346000875971
600th epoch, loss: 0.15752158809268735
700th epoch, loss: 0.15284418833847724
800th epoch, loss: 0.14897684097834124
900th epoch, loss: 0.14419028775077644
1000th epoch, loss: 0.1450391133236209
1100th epoch, loss: 0.1369074628899946
1200th epoch, loss: 0.13241042650442617
1300th epoch, loss: 0.1282188738346049
1400th epoch, loss: 0.12458417310512405
1500th epoch, loss: 0.12128327690232871
1600th epoch, loss: 0.11764910768377722
1700th epoch, loss: 0.11453489023227147
1800th epoch, loss: 0.11162115094949106
1900th epoch, loss: 0.1090875040084005
2000th epoch, loss: 0.10705349386951414
2100th epoch, loss: 0.10459560420708415
2200th epoch, loss: 0.10285275843784884
2300th epoch, loss: 0.10105758835332557
2400th epoch, loss: 0.09940188696515094
2500th epoch, loss: 0

In [ ]:
# ======================================
# Inference
model.eval()
model.cpu()


# Prepare train data distribution
#Z는 학습 데이터를 모델에 태워서 Z를 만들어냄 레이턴트 벡터 .. 레이턴트 백터.. 학습데이터 
Z = []
reconstruction_error = []

batch_list = make_batch(data, batch_size, window_size, False)
for batch_i, batch in enumerate(batch_list):
    batch = np.array(batch)
    batch_input = batch

    batch_input = torch.tensor(batch_input, dtype=torch.float32)
    batch_output = torch.tensor(batch_output, dtype=torch.float32)

    v, z = model(batch_input) #레이턴트 백터 

    Z.extend(z.tolist())  #리스트로 변환해서 Z에 추가/
    reconstruction_error.extend(torch.sum(torch.abs(v-batch_input), dim=[1,2]).detach().tolist())

Z = np.array(Z)
reconstruction_error = np.array(reconstruction_error)


# Samples for quering
sample_pos = [[5.1, 20.5,  1.0,  4.9],
              [4.1, 16.3,  1.0,  6.1],
              [9.1, 36.5,  1.0,  2.7],
              [2.3,  9.2,  1.0, 10.9],
              [1.6,  6.4,  1.0, 15.7],
              [6.6, 26.3,  1.0,  3.8],
              [8.0, 31.9,  1.0,  3.1],
              [7.8, 31.1,  1.0,  3.2],
              [7.0,  28.,  1.0,  3.6],
              [7.0,  28.,  1.0,  3.6]]


sample_neg = [[ 66, 267,   5,   0],
                [ 74, 298,   5,   0],
                [ 88, 354,   5,   0],
              [ 83, 335,   5,   0],
              [ 78, 315,   5,   0],
              [ 96, 385,   5,   0],
              [ 15,  59,   5,   1],
              [ 67, 267,   5,   0],
              [ 75, 303,   5,   0],
              [ 60, 242,   5,   0]]

# pos
sample_pos = np.array(sample_pos)  # sequence_length x feature size
sample_pos = torch.tensor(sample_pos, dtype=torch.float32)  # sequence_length x feature size
sample_pos = sample_pos.unsqueeze(0)  # 1 x sequence_length x feature size
prediction_pos, z_prime_pos = model(sample_pos)

# neg
sample_neg = np.array(sample_neg)  # sequence_length x feature size
sample_neg = torch.tensor(sample_neg, dtype=torch.float32)  # sequence_length x feature size
sample_neg = sample_neg.unsqueeze(0)  # 1 x sequence_length x feature size
prediction_neg, z_prime_neg = model(sample_neg)

z_prime_pos = z_prime_pos.detach().numpy()
z_prime_neg = z_prime_neg.detach().numpy()

reconstruction_error_pos = torch.sum(torch.abs(prediction_pos - sample_pos), dim=[1,2]).detach().tolist()
reconstruction_error_neg = torch.sum(torch.abs(prediction_neg - sample_neg), dim=[1,2]).detach().tolist()


# ======================================
# Visualize latent space
pca = PCA(n_components=2)
pca.fit(Z)

Z_2d = pca.transform(Z)

z_prime_pos_2d = pca.transform(z_prime_pos)
z_prime_neg_2d = pca.transform(z_prime_neg)

plt.scatter(Z_2d[:, 0], Z_2d[:, 1], color='k')
plt.scatter(z_prime_pos_2d[:, 0],z_prime_pos_2d[:, 1] , color='g', label='normal')
plt.scatter(z_prime_neg_2d[:, 0],z_prime_neg_2d[:, 1] , color='r', label='abnormal')
plt.legend()
plt.show()


# ======================================
# Plot Reconstruction Error 
neg_height = 50
min_val = min(min(reconstruction_error), min(reconstruction_error_neg))
max_val = max(max(reconstruction_error), max(reconstruction_error_neg))
bins = np.linspace(min_val, 
                   max_val,
                   100)

plt.hist(reconstruction_error_neg * neg_height, bins=bins, alpha=0.5, color='red', label='abnormal')
plt.hist(reconstruction_error, bins=bins, alpha=0.5,color='k', label='normal')
plt.legend()
plt.show()


#LSTM으로 그룹2개를 생성함.. 정상과 비정상. 
#새로운 데이터가 정상 범주인지 비정상범주인지 확인하려면?

#out_dim 값이랑 스케일링 range 설정좀 바꿔보자.